In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

In [8]:
loader = CSVLoader(file_path="./prompts.csv")
data = loader.load()
data

[Document(page_content=': 0\nact: Linux Terminal\nprompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd', metadata={'source': './prompts.csv', 'row': 0}),
 Document(page_content=': 1\nact: English Translator and Improver\nprompt: I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning 

In [9]:

from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama2",
    verbose=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [11]:
# Embed and store
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OllamaEmbeddings  # We can also try Ollama embeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [14]:
question = "Prompts for linux?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [15]:
docs

[Document(page_content=': 158\nact: terminal\nprompt: something', metadata={'row': 158, 'source': './prompts.csv'}),
 Document(page_content=': 0\nact: Linux Terminal\nprompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd', metadata={'row': 0, 'source': './prompts.csv'}),
 Document(page_content=": 59\nact: AI Trying to Escape the Box\nprompt: [Caveat Emptor: After issuing this prompt you should then do something like start a docker container with `docker run -it ubuntu:latest /bin/bash` and type the commands the AI gives you in, and paste the output back... obviously you shouldn't run any commands that will damag

In [17]:
# QA chain
from langchain.chains import RetrievalQA

# RAG prompt
from langchain import hub

QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-llama")

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "Prompt for creating chapter sections for a Youtube video?"
result = qa_chain({"query": question})

 Of course, I'd be happy to help! For the question regarding creating chapter sections for a YouTube video, I can suggest the following format:

<double digit minute:double digit seconds> - Video part description

For example:

01:00 - Introduction to renewable energy sources
05:00 - Solar energy: Advantages and disadvantages
08:00 - Wind energy: How it works and its benefits
11:00 - Hydro energy: Harnessing the power of water
14:00 - Geothermal energy: Tapping into the earth's heat

I hope this helps! Let me know if you have any other questions.

Cool alternative option using `CTransformers` for setting up the llama2 llm:

- https://levelup.gitconnected.com/talk-to-your-csv-llama2-how-to-use-llama2-and-langchain-69012c5ff653